In [16]:
with open("stored_text/romeo_and_juliet.txt", "r", encoding="utf-8") as file:
    text = file.read()


In [17]:
from chunking_evaluation.chunking import ClusterSemanticChunker

In [18]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint="https://ai-asksulaimanov7666ai665127629473.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15",
    api_key="6RHqiKKJ4IlJNFHJu5KvBmb50LyaLREQuezH29sDYxNkhZKWdhevJQQJ99BEACfhMk5XJ3w3AAAAACOGhZNL"
)
ef = embeddings.embed_documents

In [19]:
sem_chunk = ClusterSemanticChunker(embedding_function=ef, max_chunk_size = 150)

In [20]:
text[11000:34000]

'engthens Romeo’s hours?\n\nROMEO.\nNot having that which, having, makes them short.\n\nBENVOLIO.\nIn love?\n\nROMEO.\nOut.\n\nBENVOLIO.\nOf love?\n\nROMEO.\nOut of her favour where I am in love.\n\nBENVOLIO.\nAlas that love so gentle in his view,\nShould be so tyrannous and rough in proof.\n\nROMEO.\nAlas that love, whose view is muffled still,\nShould, without eyes, see pathways to his will!\nWhere shall we dine? O me! What fray was here?\nYet tell me not, for I have heard it all.\nHere’s much to do with hate, but more with love:\nWhy, then, O brawling love! O loving hate!\nO anything, of nothing first create!\nO heavy lightness! serious vanity!\nMisshapen chaos of well-seeming forms!\nFeather of lead, bright smoke, cold fire, sick health!\nStill-waking sleep, that is not what it is!\nThis love feel I, that feel no love in this.\nDost thou not laugh?\n\nBENVOLIO.\nNo coz, I rather weep.\n\nROMEO.\nGood heart, at what?\n\nBENVOLIO.\nAt thy good heart’s oppression.\n\nROMEO.\nWhy such 

In [21]:
sem_chunk = sem_chunk.split_text(text)

In [24]:
len(sem_chunk)

524

In [25]:
for i in range(0, 20):
    print(sem_chunk[400 + i])
    print("---------------------------")

O love! O life! Not life, but love in death! CAPULET.
Despis’d, distressed, hated, martyr’d, kill’d.
Uncomfortable time, why cam’st thou now
To murder, murder our solemnity? O child! O child! My soul, and not my child,
Dead art thou. Alack, my child is dead,
And with my child my joys are buried.
---------------------------
FRIAR LAWRENCE.
Peace, ho, for shame. Confusion’s cure lives not
In these confusions. Heaven and yourself
Had part in this fair maid, now heaven hath all, And all the better is it for the maid.
Your part in her you could not keep from death,
But heaven keeps his part in eternal life.
The most you sought was her promotion, For ’twas your heaven she should be advanc’d,
And weep ye now, seeing she is advanc’d
Above the clouds, as high as heaven itself?
---------------------------
O, in this love, you love your child so ill
That you run mad, seeing that she is well.
She’s not well married that lives married long,
But she’s best married that dies married young.
----------

In [26]:
import chromadb

client = chromadb.HttpClient(host="localhost", port=8000)
 

In [27]:
from chromadb.utils import embedding_functions
# from dotenv import dotenv_values

# config = dotenv_values(".env")
azure_openai_ef = embedding_functions.OpenAIEmbeddingFunction(

    api_base="https://ai-asksulaimanov7666ai665127629473.openai.azure.com",
    deployment_id="text-embedding-3-small",
    api_key="6RHqiKKJ4IlJNFHJu5KvBmb50LyaLREQuezH29sDYxNkhZKWdhevJQQJ99BEACfhMk5XJ3w3AAAAACOGhZNL",
    api_type="azure",
    api_version="2023-05-15", 
    model_name="text-embedding-3-small"  
)

In [ ]:
client.delete_collection

In [29]:
collection = client.get_or_create_collection("romeo_and_juliet_local")

In [30]:
collection.count()

0

In [31]:
local_ef = collection._embedding_function

In [32]:
local_ef([sem_chunk[432]])

[array([-1.64063815e-02,  8.68995562e-02,  5.37863839e-03, -1.21133421e-02,
         1.03570288e-02,  1.99228972e-02,  1.20748229e-01, -3.38747203e-02,
         6.52876422e-02, -2.05825716e-02,  1.31378807e-02,  1.97666138e-02,
        -4.79033217e-02, -8.48721638e-02, -4.70138453e-02,  2.41873022e-02,
        -5.45976683e-04, -1.29242870e-03, -1.83223821e-02,  5.54241575e-02,
         3.73355709e-02,  8.05735290e-02,  3.49711776e-02, -4.55937870e-02,
        -6.02826625e-02,  1.78425927e-02, -5.89282960e-02, -2.89410111e-02,
        -1.40756806e-02, -6.58225119e-02,  2.31585675e-03,  3.21940035e-02,
        -9.92374197e-02, -3.49735208e-02, -5.40383086e-02,  9.22371596e-02,
        -4.92838919e-02, -6.90269098e-02,  6.85826089e-05,  7.09916931e-03,
         6.65096304e-05,  3.15645300e-02, -1.61537565e-02, -2.49875933e-02,
        -5.39905243e-02,  3.15691270e-02, -3.66610521e-03,  2.20324993e-02,
         3.15077193e-02, -7.53780976e-02,  2.54654605e-02, -1.67907365e-02,
         6.3

In [33]:
from tqdm import tqdm
embeds = []
for chunk in tqdm(sem_chunk):
    embeds.append(local_ef([chunk])[0])


100%|██████████| 524/524 [04:18<00:00,  2.03it/s]


In [37]:
collection.add(documents=sem_chunk, ids = [f"i{i}" for i in range(len(sem_chunk))], embeddings=embeds)

In [38]:
collection.count()

524

In [39]:
collection.query(query_texts=["Why is Romeo sad at the beginning of the play?"])

{'ids': [['i167',
   'i292',
   'i411',
   'i156',
   'i478',
   'i303',
   'i158',
   'i214',
   'i121',
   'i120']],
 'distances': [[0.7090871,
   0.7141158,
   0.75601256,
   0.7649276,
   0.7783615,
   0.78697675,
   0.80004406,
   0.80978894,
   0.81879306,
   0.8267069]],
 'embeddings': None,
 'metadatas': [[None, None, None, None, None, None, None, None, None, None]],
 'documents': [['ROMEO.\nO let us hence; I stand on sudden haste.\n\nFRIAR LAWRENCE.\nWisely and slow; they stumble that run fast.\n\n [_Exeunt._]\n\nSCENE IV. A Street. Enter Benvolio and Mercutio.\n\nMERCUTIO.\nWhere the devil should this Romeo be? Came he not home tonight?\n\nBENVOLIO.\nNot to his father’s; I spoke with his man. MERCUTIO.\nWhy, that same pale hard-hearted wench, that Rosaline, torments him so\nthat he will sure run mad.',
   'ROMEO.\nSpakest thou of Juliet? How is it with her?\nDoth not she think me an old murderer,\nNow I have stain’d the childhood of our joy With blood remov’d but little from 

In [ ]:
len(embeds)

In [ ]:
print(sem_chunk[432])

In [24]:
collection.add(documents=sem_chunk, ids = [f"i{i}" for i in range(len(sem_chunk))], embeddings=embeds)

In [ ]:
collection

In [27]:
local_ef = collection._embedding_function

In [ ]:
collection.count()

In [ ]:
results = collection.get(ids=[f"i{i}" for i in range(len(sem_chunk))])
print(results)